In [28]:
# !pip install pyomo
# !pip install idaes-pse

In [29]:

import numpy as np
import pandas as pd
# from contextlib import redirect_stdout

from pyomo.environ import *


In [30]:
# import os

# !idaes get-extensions --to ./bin
# os.environ['PATH'] += ':bin'

Getting files...
Done
---------------------------------------------------
IDAES Extensions Build Versions
Solvers:  v3.4.0 20230607-2153 windows-x86_64
Library:  v3.4.0 20230607-2236 windows-x86_64


In [31]:

sig = 0

UTEs = [('UTE1', 'UTE2', 'UTE3', 'UTE4'),
        (1000, 970, 700, 680),
        (16.19, 17.26, 16.6, 16.5),
        (0.00048, 0.00031, 0.002, 0.00211),
        (455, 455, 130, 130),
        (150, 150, 20, 20)]
UTEs = np.transpose(UTEs)
UTE = pd.DataFrame(UTEs, columns=['Nome','a','b','c','PGmax','PGmin'])



Dem = [('Hora1', 450, 45),
       ('Hora2', 530, 53),
       ('Hora3', 600, 60),
       ('Hora4', 540, 54),
       ('Hora5', 400, 40),
       ('Hora6', 280, 28),
       ('Hora7', 290, 29),
       ('Hora8', 500, 50)]
Dem = pd.DataFrame(Dem, columns=['Hora','Dem', 'Res'])

UTE['a'] = UTE['a'].astype('float')
UTE['b'] = UTE['b'].astype('float')
UTE['c'] = UTE['c'].astype('float')
UTE['PGmax'] = UTE['PGmax'].astype('float')
UTE['PGmin'] = UTE['PGmin'].astype('float')


In [36]:

def Otimiza_UC(UTE,Dem,Sig):
    #=====================================Modelagem do Problema============================================================
    modelo = ConcreteModel('Otimizacao Unnit Commitment')

    def limite_UTE(modelo,pg,ext):
        return (UTE[UTE['Nome'] == pg]['PGmin'].values[0], UTE[UTE['Nome'] == pg]['PGmax'].values[0])
    def limite_U(modelo,pg,ext):
        return (0, 1)
    def limite_X(modelo,pg,ext):
        return (0.0, 20.0)

    # ----------------    Declarar Variáveis
    modelo.Pg = Var(UTE['Nome'], Dem['Hora'], bounds=limite_UTE, domain=NonNegativeReals)
    modelo.U = Var(UTE['Nome'], Dem['Hora'], bounds=limite_U, domain=NonNegativeReals) # Função Decisão de Operação - FDO

    if Sig == 1:
        modelo.X = Var(UTE['Nome'], Dem['Hora'], bounds=limite_X, domain=NonNegativeReals) # Argumento da FDO  -  utilizado na aproximação sigmoidal


    # ----------------    Declarar Função Objetivo
    def objective_rule(modelo):
        fob = 0
        for h in Dem['Hora']:
            fob += sum((UTE[UTE['Nome'] == g]['a'].values[0] +
                        UTE[UTE['Nome'] == g]['b'].values[0]*modelo.Pg[g,h] +
                        UTE[UTE['Nome'] == g]['c'].values[0]*modelo.Pg[g,h]**2)*modelo.U[g,h] for g in UTE['Nome'])
        return fob


    modelo.objetivo = Objective(rule=objective_rule,sense=minimize)

    # ----------------     Declarar Restrições
    modelo.AtendDem = ConstraintList()
    modelo.AtendDemRes = ConstraintList()
    modelo.LimGer = ConstraintList()
    # modelo.Sigmoide = ConstraintList()


    for h in Dem['Hora']:
        modelo.AtendDem.add(sum(modelo.U[g,h]*modelo.Pg[g,h] for g in UTE['Nome']) == Dem[Dem['Hora'] == h]['Dem'].values[0])
    for h in Dem['Hora']:
        modelo.AtendDemRes.add(sum(modelo.U[g,h]*UTE[UTE['Nome'] == g]['PGmax'].values[0] for g in UTE['Nome']) >=
                               Dem[Dem['Hora'] == h]['Dem'].values[0] + Dem[Dem['Hora'] == h]['Res'].values[0])
    # for h in Dem['Hora']:
    #     for g in UTE['Nome']:
    #         modelo.LimGer.add(UTE[UTE['Nome'] == g]['PGmin'].values[0] <= modelo.Pg[g,h])
    #         modelo.LimGer.add(modelo.Pg[g,h] <= UTE[UTE['Nome'] == g]['PGmax'].values[0])



    # FDO Sigmoidal - definição

    if Sig == 1:
        modelo.FDO_Sigmoide = ConstraintList() # Restrição da FDO Sigmoidal
        alpha=.5
        for h in Dem['Hora']:
            for g in UTE['Nome']:
                modelo.FDO_Sigmoide.add(modelo.U[g,h]  -  (exp(alpha * (modelo.X[g,h])) - 1) / (exp(alpha * (modelo.X[g,h])) + 1) == 0)


    # caso Sig == 0, o algoritmo decidirá o valor de U entre 0 e 1; logo, será feita uma aproximação linear




    #------------------------- Adição da aproximação pela função sigmoide
    # for h in Dem['Hora']:
    #     for g in UTE['Nome']:
    #         modelo.Sigmoide.add(0 <= 1/(1+ exp(-modelo.U[g,h])))
    #         modelo.Sigmoide.add(1/(1+ exp(-modelo.U[g,h])) <= 1)


    # if sig == 0:
    #     modelo.Sigmoide.deactivate()



    # with open('resumo_modelo.txt', 'w') as f:
    #     # Redirecione a saída para o arquivo
    #     with redirect_stdout(f):
    #         # Imprima um resumo do modelo
    #         modelo.pprint()

    # modelo.pprint()



    ### Executar a Otimização.
    # solver = SolverFactory('ipopt')

    solver = SolverFactory('ipopt',executable='bin/ipopt')
    resultados_seq = solver.solve(modelo, tee=False)


    # resultados_seq = solver.solve(model_seq, tee=False)
    # resultados_con = solver.solve(model_con, tee=False)

    # Relatório dos resultados de otimização

    print('Status Final do Problema de Otimização:', resultados_seq.solver.status)
    print('Condição de Término:', resultados_seq.solver.termination_condition)
    print('Resultado Função Objetivo:', value(modelo.objetivo), '\n')
    return modelo


In [37]:

modelo_arred = Otimiza_UC(UTE, Dem, 0)
modelo_sig = Otimiza_UC(UTE, Dem, 1)


Status Final do Problema de Otimização: ok
Condição de Término: optimal
Resultado Função Objetivo: 67863.08835063892 

Status Final do Problema de Otimização: ok
Condição de Término: optimal
Resultado Função Objetivo: 67863.25264385746 



In [73]:

U_sig = modelo_sig.U.extract_values()
X_sig = modelo_sig.X.extract_values()

Arred = modelo_arred.Pg.extract_values()
Sigm = modelo_sig.Pg.extract_values()

# print(Arred, '\n')
# print(Sigm, '\n')
# print(U_sig, '\n')
# print(X_sig, '\n')

# for i in modelo_arred.Pg.extract_values():
#     Arred += modelo_arred.Pg[i].value
#     print(i)

